In [1]:
import pandas as pd
import numpy as np
import datetime

## Setup some fake data

In [2]:
def generate_random_date():
    return datetime.datetime(
        np.random.randint(1999, 2015),
        np.random.randint(1, 12),
        np.random.randint(1, 28),
        np.random.randint(0, 23),
        np.random.randint(0, 59),
        np.random.randint(0, 59),
    )

In [3]:
num_rows = 10
data = pd.DataFrame({
        'from': np.random.randint(0, 100, num_rows),
        'to': np.random.randint(0, 100, num_rows),
        'time': [generate_random_date() for _ in range(num_rows)],
        'duration': np.random.exponential(30, num_rows),
    })

In [4]:
resample_indices = np.random.randint(0, num_rows-1, 30)
data_with_duplicates = data.ix[resample_indices]

In [5]:
data_with_duplicates_jittered = data_with_duplicates.copy()
data_with_duplicates_jittered['time'] = data_with_duplicates_jittered['time'] + \
    [datetime.timedelta(0,np.random.randint(0,2)) 
     for _ in range(data_with_duplicates.shape[0])]

## Remove duplicate records

A duplicate record is defined as having same from, to, and time. Remove duplicates using the pandas function

In [6]:
filtered_method_one = data_with_duplicates.drop_duplicates()

If duplicates are a bit fuzzier, i.e. a sequence of records within 2 seconds, it is a bit more complicated.

Sort by from, to, time

Then calculate groups/windows by finding chains of records with time within 2 seconds

In [7]:
def to_seconds(x):
    try:
        return x.seconds
    except:
        pass

In [8]:
dwdj_sorted = data_with_duplicates_jittered.sort(['from', 'to', 'time'])
dwdj_sorted['time_since_previous'] = dwdj_sorted['time'] - dwdj_sorted['time'].shift(1) 
dwdj_sorted['time_since_previous'] = dwdj_sorted['time_since_previous'].apply(lambda x: to_seconds(x))

/home/joth/projects/2016-11-24_pandas-cobrien/venv/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [9]:
dwdj_sorted.head(5)

,duration,from,time,to,time_since_previous
2,25.054552,9,2007-08-18 13:12:56,76,NaN
2,25.054552,9,2007-08-18 13:12:57,76,1.0
2,25.054552,9,2007-08-18 13:12:57,76,0.0
2,25.054552,9,2007-08-18 13:12:57,76,0.0
3,18.737299,10,2004-10-18 03:14:44,30,50507.0


Now, we can remove rows if they are not the first in a from,to pair or have a time_since_previous of smaller than 2 seconds.

Can't just remove ones which are smaller than 2 seconds, as they may not have same from, to values.

In [10]:
duplicate_from_to = (
    (dwdj_sorted['from'] == dwdj_sorted['from'].shift(1)) \
    & (dwdj_sorted['to'] == dwdj_sorted['to'].shift(1))
)
first_record = ~duplicate_from_to

In [11]:
sel = first_record | (dwdj_sorted['time_since_previous'] > 2)

In [12]:
sel & dwdj_sorted['time_since_previous'] > 2

2    False
2    False
2    False
2    False
3    False
3    False
3    False
8    False
8    False
7    False
7    False
7    False
7    False
7    False
1    False
1    False
1    False
1    False
5    False
5    False
5    False
5    False
0    False
0    False
0    False
0    False
6    False
4    False
4    False
4    False
dtype: bool

In [13]:
dwdj_sorted[sel]

,duration,from,time,to,time_since_previous
2,25.054552,9,2007-08-18 13:12:56,76,NaN
3,18.737299,10,2004-10-18 03:14:44,30,50507.0
8,6.997534,20,2014-05-10 01:38:40,17,80636.0
7,1.792194,26,2007-07-03 07:26:12,73,20852.0
1,54.496066,42,2007-09-07 09:35:45,38,7772.0
5,41.127511,48,2013-07-18 22:33:33,10,46667.0
0,7.641597,53,2001-06-21 05:27:58,97,24864.0
6,34.137307,77,1999-06-07 19:20:58,21,49979.0
4,32.226882,95,2011-07-13 05:42:36,24,37298.0


This is not bulletproof. There is a tradeoff between the tolerance (2 seconds) and the possibility of error. In particular, a larger tolerance will omit records which should in fact not be omitted. The appropriate tolerance is probalby the variability/noise in the time records of the hardware. If that is not known a good way to start validate would be to use various tolerances, summing the durations for each. Unambigious records can be removed, and the variability compared to try and see if there are any inflections/outliers that indicate a poor tolerance.

Keep a record like so

| duration | tolerance |  
| ---- | ---- |
|     3243253 | 1 |
| 3243253 | 2 |
| 3243253 | 3 |
| 3243345 | 4 |